In [1]:
# !pip install spinesTS
import sys
sys.path.append('..')

In [2]:
import torch
from spinesTS.layers import TrainableMovingAverage1d

a = torch.randn((32, 64))
TrainableMovingAverage1d(3)(a).shape

torch.Size([32, 64])

# MLPTorchModel

In [3]:
from spinesTS.preprocessing import split_series
from spinesTS.plotting import plot2d
from sklearn.metrics import r2_score

In [4]:
from spinesTS.data import BuiltInSeriesData
from spinesTS.nn import *
from spinesTS.metrics import mean_absolute_error
from spinesTS.metrics import mean_absolute_percentage_error

In [5]:
series_data = BuiltInSeriesData()

+----+----------------------+----------------------------------------------+
|    | table's name         | table's columns                              |
+----+----------------------+----------------------------------------------+
|  0 | ETTh1                | date, HUFL, HULL, MUFL, MULL, LUFL, LULL, OT |
|  1 | ETTh2                | date, HUFL, HULL, MUFL, MULL, LUFL, LULL, OT |
|  2 | ETTm1                | date, HUFL, HULL, MUFL, MULL, LUFL, LULL, OT |
|  3 | ETTm2                | date, HUFL, HULL, MUFL, MULL, LUFL, LULL, OT |
|  4 | Electric_Production  | date, value                                  |
|  5 | Messages_Sent        | date, ta, tb, tc                             |
|  6 | Messages_Sent_Hour   | date, hour, ta, tb, tc                       |
|  7 | Series_0             | date, timepoint, turnover_1, turnover_2      |
|  8 | Series_1             | date, timepoint, turnover_1, turnover_2      |
|  9 | Series_2             | date, timepoint, turnover_1, turnover_2      |

In [6]:
cs = series_data[0]
cs_data = cs.dataset
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
data = mms.fit_transform(cs_data.iloc[:, 1:].values)

x_train, x_test, y_train, y_test = split_series(data, cs_data['OT'], 48, 24, train_size=0.9)

In [7]:
x_train.shape, x_test.shape

((15614, 48, 7), (1735, 48, 7))

## RecurrentWeightedDenseNet

### one res_dense_blocks

In [ ]:
wdr = Time2VecNet((48, 7), 24, random_seed=42, flip_features=False, learning_rate=0.01)
wdr.fit(x_train, y_train, eval_set=(x_test[:-2], y_test[:-2]), batch_size=32,
             min_delta=0, patience=100, epochs=1000, verbose=True, lr_scheduler='ReduceLROnPlateau')
y_pred_cs = wdr.predict(x_test[-2:])
print(f"r2: {r2_score(y_test[-2:].T, y_pred_cs.T)}")
print(f"mae: {mean_absolute_error(y_test[-2:], y_pred_cs)}")
print(f"mape: {mean_absolute_percentage_error(y_test[-2:], y_pred_cs)}")
a = plot2d(y_test[-2:], y_pred_cs, eval_slices='[-1]', labels=['y_test', 'y_pred'], figsize=(20, 6))

Epoch 1/1000 
488/488 - p0 - loss: 0.5716 - mae: 3.1363 - val_loss: 3.0004 - val_mae: 3.4908 - 6.22s/epoch - 0.013s/step
Epoch 2/1000 
488/488 - p0 - loss: 0.4052 - mae: 2.7591 - val_loss: 2.2391 - val_mae: 2.7192 - 6.19s/epoch - 0.013s/step
Epoch 3/1000 
488/488 - p0 - loss: 0.4644 - mae: 3.0047 - val_loss: 1.7462 - val_mae: 2.2134 - 6.26s/epoch - 0.013s/step
Epoch 4/1000 
488/488 - p1 - loss: 0.4547 - mae: 2.9547 - val_loss: 1.7666 - val_mae: 2.2348 - 6.25s/epoch - 0.013s/step
Epoch 5/1000 
488/488 - p2 - loss: 0.4597 - mae: 2.8754 - val_loss: 1.7688 - val_mae: 2.2378 - 6.78s/epoch - 0.014s/step
Epoch 6/1000 
488/488 - p0 - loss: 0.4918 - mae: 2.8924 - val_loss: 1.6813 - val_mae: 2.1470 - 6.70s/epoch - 0.014s/step
Epoch 7/1000 
488/488 - p0 - loss: 0.4391 - mae: 2.8171 - val_loss: 1.4863 - val_mae: 1.9424 - 6.04s/epoch - 0.012s/step
Epoch 8/1000 
488/488 - p1 - loss: 0.4802 - mae: 2.7504 - val_loss: 1.7513 - val_mae: 2.2208 - 6.06s/epoch - 0.012s/step
Epoch 9/1000 
488/488 - p2 - los

In [10]:
# GAU WITHOUT FLIP
# r2: -2.461928251531084
# mae: 0.7824241320292159
# mape: 0.08264914036325263

# FLIP
# r2: -3.2216385445817077
# mae: 0.8676682909329737
# mape: 0.09149143596419052

# GAU multi variable, without flip
# r2: -0.3924809760700697
# mae: 0.5082427859306335
# mape: 0.052218739937045545

# GAU multi variable, flip
# r2: -0.15485139586619767
# mae: 0.4762745102246601
# mape: 0.0496499044584968


# Time2Vec without flip
# r2: -3.110017119085237
# mae: 0.8972663482030238
# mape: 0.09488035043723038

# Time2Vec, flip
# r2: -3.793590596311873
# mae: 1.0186231931050622
# mape: 0.10717737492764423

# Time2Vec multi variable, without flip
# r2: -0.33395022407345076
# mae: 0.5126790603001914
# mape: 0.05252923273324144

# Time2Vec multi variable, flip
# r2: -2.0952745732482994
# mae: 0.7136228879292804
# mape: 0.07433070742039964

# WeightedDenseRNN
# r2: -3.93017224579161
# mae: 0.8334025740623479
# mape: 0.08885424620348133

# WeightedDenseRNN multi variable
# r2: -0.6424794525661933
# mae: 0.5684142311414083
# mape: 0.05887982151593065